# Assignment 1

Srikar Pusuluri

U95310908

## Description of the Analysis

- The Dataset consists of 5 features and 2 classes 
- Total number of observations are 5404.
- This dataset aims to distinguish between oral (class 1) and nasal (class 0) sounds (class 1). 3,818 samples are in class 0 and 1,586 samples are in class 1.
- The phonemes are written as follows: sh as in she, dcl as in dark, iy as the vowel in she, aa as the vowel in dark, and ao as the initial vowel in water.

## Problem/Goal for your Analysis

To determine whether a sound in the dataset is oral or nasal based on the phonemes.

### Importing the required libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

np.random.seed(1)

### Load, Clean and Prepare data

In [3]:
#loading the dataset
df= pd.read_csv('phonemedataset.csv')

In [4]:
df.head(10)

,Aa,Ao,Dcl,Iy,Sh,Class
0,1.240,0.875,-0.205,-0.078,0.067,0
1,0.268,1.352,1.035,-0.332,0.217,0
2,1.567,0.867,1.300,1.041,0.559,0
3,0.279,0.990,2.555,-0.738,0.000,0
4,0.307,1.272,2.656,-0.946,-0.467,0
5,0.960,0.712,-0.143,0.091,-0.061,0
6,2.941,0.657,1.493,-0.331,-0.123,0
7,0.401,1.159,2.064,-0.690,-0.308,0
8,0.229,1.748,0.634,0.259,0.000,0
9,0.303,0.609,0.474,1.427,-0.811,1


> For better understanding of the data. Here are names of the attribute.

- Aa as the vowel in dark
- Ao as the initial vowel in water
- Dcl as in dark
- Iy as the vowel in she
- Sh as in she
- Class 0 - Nasal Sound
- Class 1 - Oral Sound

### Data Exploration

In [5]:
# Explore the dataset
print(df.columns)
print(df.describe())
print(df.info())

Index(['Aa', ' Ao', ' Dcl', ' Iy', ' Sh', 'Class'], dtype='object')
                Aa           Ao          Dcl           Iy           Sh  \
count  5404.000000  5404.000000  5404.000000  5404.000000  5404.000000   
mean      0.818957     1.258802     0.764732     0.398743     0.078619   
std       0.858733     0.851057     0.925436     0.796531     0.575624   
min      -1.700000    -1.327000    -1.823000    -1.581000    -1.284000   
25%       0.243750     0.596000    -0.115000    -0.205000    -0.232250   
50%       0.492500     1.075500     0.729000     0.285500    -0.044000   
75%       1.089250     1.866250     1.484000     0.937000     0.196250   
max       4.107000     4.378000     3.199000     2.826000     2.719000   

             Class  
count  5404.000000  
mean      0.293486  
std       0.455401  
min       0.000000  
25%       0.000000  
50%       0.000000  
75%       1.000000  
max       1.000000  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5404 entries, 0 to 5403
Da

In [6]:
# Just Tranforming the Matrix, So that we can observe all the continuous columns(rows)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Aa,5404.0,0.818957,0.858733,-1.700,0.24375,0.4925,1.08925,4.107
Ao,5404.0,1.258802,0.851057,-1.327,0.59600,1.0755,1.86625,4.378
Dcl,5404.0,0.764732,0.925436,-1.823,-0.11500,0.7290,1.48400,3.199
Iy,5404.0,0.398743,0.796531,-1.581,-0.20500,0.2855,0.93700,2.826
Sh,5404.0,0.078619,0.575624,-1.284,-0.23225,-0.0440,0.19625,2.719
Class,5404.0,0.293486,0.455401,0.000,0.00000,0.0000,1.00000,1.000


### Performing Data Cleaning

>  Here in our dataset 'class' is the target where '0' indicates Nasal Sound and '1' indicates Oral Sound


In [7]:
df.isnull().sum().sum()

0

In [8]:
df.columns = df.columns.str.lower()
df.head(5)

,aa,ao,dcl,iy,sh,class
0,1.240,0.875,-0.205,-0.078,0.067,0
1,0.268,1.352,1.035,-0.332,0.217,0
2,1.567,0.867,1.300,1.041,0.559,0
3,0.279,0.990,2.555,-0.738,0.000,0
4,0.307,1.272,2.656,-0.946,-0.467,0


In [9]:
df['class'].unique()

array([0, 1], dtype=int64)

In [10]:
df[['class']].value_counts()

class
0        3818
1        1586
dtype: int64

- Here we can see that there is an imbalance in the output class 

- 0 - 3818 instances
- 1 - 1586 instances

## Splitting the data and Standarizing the data

- Here we are splitting the droping the output of the data, so that we can create a separate dataframe for the output.
- Splitting the data into 70/30
- Fitting the scalar to the train data 
- Transforming the train and test data


In [11]:
from sklearn.preprocessing import StandardScaler
#splitting the data into X,y 
X = df.drop('class', axis=1)
y = df['class']

# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3)

# Initialize the scaler
scler = StandardScaler()

# Fitting the scaler to the training set
scler.fit(X_train)

# Transform the training and testing sets
X_train = scler.transform(X_train)
X_test = scler.transform(X_test)

- After scaling the training data.
- Applying undersampling technique to the scaled training data.

### Data Imbalancing

In [12]:
undrsample = RandomUnderSampler(sampling_strategy='majority')

X_train, y_train = undrsample.fit_resample(X_train, y_train)

print(Counter(y_train))

Counter({0: 1103, 1: 1103})


- After performing undersampling, now the output class is balanced.
- 0 - 1103
- 1 - 1103

Converting numpy array into the dataframe

In [13]:
X_train=pd.DataFrame(X_train)
y_train=pd.DataFrame(y_train)
X_test=pd.DataFrame(X_test)
y_test=pd.DataFrame(y_test)

Saving the dataframe to csv

In [14]:
X_train.to_csv('phoneme_Xtrain.csv', index=False)
X_test.to_csv('phoneme_Xtest.csv', index=False)
y_train.to_csv('phoneme_ytrain.csv', index=False)
y_test.to_csv('phoneme_ytest.csv', index=False)

Saving the files into training and test. Now we will load these csv files into the Modelfitting notebook. 